# **Heart Attack Classification**

# **1. Import libraries**

In [1]:
# Basic packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

# Scaling and model selection
from sklearn.preprocessing import MinMaxScaler
# Train test split
from sklearn.model_selection import train_test_split

# Base model
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# Ensemble model
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

import warnings
warnings.filterwarnings("ignore")

# **2. Read in the data**


# **3. Exploratory Data Analysis**